In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import nrrd
import os
import pandas as pd
import random
import json
import cv2
import math
import pydicom as dicomio

from pydicom import dcmread
from scipy import ndimage
from skimage.util import montage
from skimage.measure import label, regionprops, find_contours
from skimage.morphology import disk, binary_dilation, binary_opening

In [1]:
# Global variables
EXAMS_DIR = "/Storage/PauloOctavioDir/Exames/"

In [ ]:
def normalize(image):
    min_val = MIN_HU_VALUE
    max_val = MAX_HU_VALUE
    image[image < min_val] = min_val
    image[image > max_val] = max_val
    image = (image - min_val) / (max_val - min_val)
    return image

In [4]:
def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)
    
    # Convert to Hounsfield units (HU)
    intercept = -1024
    slope = 1
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [5]:
# Load the scans in given folder path
def load_exam(path):
    slices = [dcmread(path + '/' + s) for s in os.listdir(path) if s != 'rtss.dcm']
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    volume = get_pixels_hu(slices)  
    return volume

def load_mask(path):
    slices = [dcmread(path + '/' + s) for s in os.listdir(path) if s != 'rtss.dcm']
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    volume = np.stack([s.pixel_array for s in slices])
    return np.array(volume, dtype=np.int16)

def get_volumes(images_path, masks_path, exam_id):
    root_path = EXAMS_DIR + str(exam_id)
    exam = load_exam(root_path + images_path)
    masks = load_mask(root_path + masks_path)
    return exam, masks

In [6]:
def print_montage(volume, cmap=None, div=10):
    no_cols = np.ceil(volume.shape[0] / div).astype(int)
    fig, ax1 = plt.subplots(1, 1, figsize=(20, 10))
    ax1.imshow(montage(volume, grid_shape=(div,no_cols)), cmap)

In [9]:
# Source: https://stackoverflow.com/questions/41793931/plotting-images-side-by-side-using-matplotlib
def show_image_list(list_images, list_titles=None, list_cmaps=None, grid=False, num_cols=2, figsize=(8, 4), title_fontsize=15,
                   hide_axis=False, save_fig=False, vmin=None, vmax=None, output_path=None):
    '''
    Shows a grid of images, where each image is a Numpy array. The images can be either
    RGB or grayscale.

    Parameters:
    ----------
    images: list
        List of the images to be displayed.
    list_titles: list or None
        Optional list of titles to be shown for each image.
    list_cmaps: list or None
        Optional list of cmap values for each image. If None, then cmap will be
        automatically inferred.
    grid: boolean
        If True, show a grid over each image
    num_cols: int
        Number of columns to show.
    figsize: tuple of width, height
        Value to be passed to pyplot.figure()
    title_fontsize: int
        Value to be passed to set_title().
    hide_axis: bool
        If True, hide images axis.
    save_fig: bool
        If True, saves image list.
    vmin, vmax : scalar, optional
        When using scalar data and no explicit *norm*, *vmin* and *vmax*
        define the data range that the colormap covers. By default,
        the colormap covers the complete value range of the supplied
        data. *vmin*, *vmax* are ignored if the *norm* parameter is used.
    output_path: str
        Value to be passed to pyplot.savefig()
    '''

    assert isinstance(list_images, list)
    assert len(list_images) > 0
    assert isinstance(list_images[0], np.ndarray)

    if list_titles is not None:
        assert isinstance(list_titles, list)
        assert len(list_images) == len(list_titles), '%d imgs != %d titles' % (len(list_images), len(list_titles))

    if list_cmaps is not None:
        assert isinstance(list_cmaps, list)
        assert len(list_images) == len(list_cmaps), '%d imgs != %d cmaps' % (len(list_images), len(list_cmaps))

    num_images  = len(list_images)
    num_cols    = min(num_images, num_cols)
    num_rows    = int(num_images / num_cols) + (1 if num_images % num_cols != 0 else 0)

    # Create a grid of subplots.
    fig, axes = plt.subplots(num_rows, num_cols, figsize=figsize)
    
    # Create list of axes for easy iteration.
    if isinstance(axes, np.ndarray):
        list_axes = list(axes.flat)
    else:
        list_axes = [axes]

    for i in range(num_images):

        img    = list_images[i]
        title  = list_titles[i] if list_titles is not None else None
        cmap   = list_cmaps[i] if list_cmaps is not None else None
        
        list_axes[i].imshow(img, cmap=cmap, vmin=None, vmax=None)
        list_axes[i].set_title(title, fontsize=title_fontsize) 
        list_axes[i].grid(grid)

    for i in range(num_images, len(list_axes)):
        list_axes[i].set_visible(False)
    
    if hide_axis:
        for i in range(len(list_axes)):
            list_axes[i].set_axis_off()
        
    fig.tight_layout()
    
    if save_fig:
        plt.savefig(output_path)
    _ = plt.show()

### Nodules RoI

In [ ]:
def crop_roi(image, left, top, right, bottom):
    croped_image = image[top:bottom,
                left:right]
    return croped_image

def mask_to_bbox(mask):
    # ref: https://github.com/nikhilroxtomar/Semantic-Segmentation-Mask-to-Bounding-Box/blob/main/mask_to_bbox.py
    lbl = label(mask)
    props = regionprops(lbl)
    if len(props) == 0:
        raise ValueError('No mask identified.')
    if len(props) == 1:
        prop = props[0]
    else:
        areas = [r.area for r in props]
        areas.sort()
        for region in props:
            if region.area == areas[-1]:
                prop = region
    x1 = prop.bbox[1]
    y1 = prop.bbox[0]
    x2 = prop.bbox[3]
    y2 = prop.bbox[2]
    bbox = [x1, y1, x2, y2]
    return bbox

def mask_to_roi(mask, image, bbox_extension=0):
    
    bboxes = mask_to_bbox(mask)
    cropped_image = crop_roi(
        image, 
        bboxes[0] - bbox_extension, 
        bboxes[1] - bbox_extension, 
        bboxes[2] + bbox_extension, 
        bboxes[3] + bbox_extension
    )
    
    return cropped_image

def get_pixel_array(dicom_file):
    data = dcmread(dicom_file)
    return data.pixel_array

## DICOM

In [ ]:
import pydicom
from pydicom.dataset import Dataset, FileDataset
from pydicom.uid import ExplicitVRLittleEndian
import pydicom._storage_sopclass_uids

def write_dicom(image, filename, rescale_intercept="0", rescale_slope="1", pixel_spacing=r"1\1"): 
    # ref: https://stackoverflow.com/questions/14350675/create-pydicom-file-from-numpy-array
    if image.dtype != np.uint16:
        image = image.astype(np.uint16)
        
    meta = pydicom.Dataset()
    meta.MediaStorageSOPClassUID = pydicom._storage_sopclass_uids.CTImageStorage
    meta.MediaStorageSOPInstanceUID = pydicom.uid.generate_uid()
    meta.TransferSyntaxUID = pydicom.uid.ExplicitVRLittleEndian  

    ds = Dataset()
    ds.file_meta = meta

    ds.is_little_endian = True
    ds.is_implicit_VR = False

    ds.SOPClassUID = pydicom._storage_sopclass_uids.CTImageStorage

    ds.Modality = "CT"
    ds.SeriesInstanceUID = pydicom.uid.generate_uid()
    ds.StudyInstanceUID = pydicom.uid.generate_uid()
    ds.FrameOfReferenceUID = pydicom.uid.generate_uid()

    ds.BitsStored = 16
    ds.BitsAllocated = 16
    ds.SamplesPerPixel = 1
    ds.HighBit = 15

    ds.ImagesInAcquisition = "1"

    ds.Rows = image.shape[0]
    ds.Columns = image.shape[1]
    ds.InstanceNumber = 1
    
    ds.RescaleIntercept = rescale_intercept
    ds.RescaleSlope = rescale_slope
    ds.PixelSpacing = pixel_spacing
    ds.PhotometricInterpretation = "MONOCHROME2"
    ds.PixelRepresentation = 1

    pydicom.dataset.validate_file_meta(ds.file_meta, enforce_standard=True)
        
    ds.PixelData = image.tobytes()
    ds.save_as(filename, write_like_original=False)
    
def show_dicom(dicom_file):
    image = get_pixel_array(dicom_file)
    plt.imshow(image)
    plt.show()

## GAN

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

factors = [1, 1, 1, 1, 1 / 2, 1 / 4, 1 / 8, 1 / 16, 1 / 32]

class WSConv2d(nn.Module):
    """
    Weight scaled Conv2d (Equalized Learning Rate)
    Note that input is multiplied rather than changing weights
    this will have the same result.

    Inspired and looked at:
    https://github.com/nvnbny/progressive_growing_of_gans/blob/master/modelUtils.py
    """

    def __init__(
        self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, gain=2
    ):
        super(WSConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.scale = (gain / (in_channels * (kernel_size ** 2))) ** 0.5
        self.bias = self.conv.bias
        self.conv.bias = None

        # initialize conv layer
        nn.init.normal_(self.conv.weight)
        nn.init.zeros_(self.bias)

    def forward(self, x):
        return self.conv(x * self.scale) + self.bias.view(1, self.bias.shape[0], 1, 1)


class PixelNorm(nn.Module):
    def __init__(self):
        super(PixelNorm, self).__init__()
        self.epsilon = 1e-8

    def forward(self, x):
        return x / torch.sqrt(torch.mean(x ** 2, dim=1, keepdim=True) + self.epsilon)


class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, use_pixelnorm=True):
        super(ConvBlock, self).__init__()
        self.use_pn = use_pixelnorm
        self.conv1 = WSConv2d(in_channels, out_channels)
        self.conv2 = WSConv2d(out_channels, out_channels)
        self.leaky = nn.LeakyReLU(0.2)
        self.pn = PixelNorm()

    def forward(self, x):
        x = self.leaky(self.conv1(x))
        x = self.pn(x) if self.use_pn else x
        x = self.leaky(self.conv2(x))
        x = self.pn(x) if self.use_pn else x
        return x


class Generator(nn.Module):
    def __init__(self, z_dim, in_channels, img_channels=3):
        super(Generator, self).__init__()

        # initial takes 1x1 -> 4x4
        self.initial = nn.Sequential(
            PixelNorm(),
            nn.ConvTranspose2d(z_dim, in_channels, 4, 1, 0),
            nn.LeakyReLU(0.2),
            WSConv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2),
            PixelNorm(),
        )

        self.initial_rgb = WSConv2d(
            in_channels, img_channels, kernel_size=1, stride=1, padding=0
        )
        self.prog_blocks, self.rgb_layers = (
            nn.ModuleList([]),
            nn.ModuleList([self.initial_rgb]),
        )

        for i in range(
            len(factors) - 1
        ):  # -1 to prevent index error because of factors[i+1]
            conv_in_c = int(in_channels * factors[i])
            conv_out_c = int(in_channels * factors[i + 1])
            self.prog_blocks.append(ConvBlock(conv_in_c, conv_out_c))
            self.rgb_layers.append(
                WSConv2d(conv_out_c, img_channels, kernel_size=1, stride=1, padding=0)
            )

    def fade_in(self, alpha, upscaled, generated):
        # alpha should be scalar within [0, 1], and upscale.shape == generated.shape
        return torch.tanh(alpha * generated + (1 - alpha) * upscaled)

    def forward(self, x, alpha, steps):
        out = self.initial(x)

        if steps == 0:
            return self.initial_rgb(out)

        for step in range(steps):
            upscaled = F.interpolate(out, scale_factor=2, mode="nearest")
            out = self.prog_blocks[step](upscaled)

        # The number of channels in upscale will stay the same, while
        # out which has moved through prog_blocks might change. To ensure
        # we can convert both to rgb we use different rgb_layers
        # (steps-1) and steps for upscaled, out respectively
        final_upscaled = self.rgb_layers[steps - 1](upscaled)
        final_out = self.rgb_layers[steps](out)
        return self.fade_in(alpha, final_upscaled, final_out)
    
def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location="cuda")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

In [8]:
import png
import torchvision.transforms as transforms

def denormalize(image):
    min_val = MIN_HU_VALUE
    max_val = MAX_HU_VALUE
    return image * (max_val - min_val) + min_val

def generate_images(gen, step, n, hist_type, path):
    with torch.no_grad():
        for i in range(n):
            noise = torch.randn(1, Z_DIM, 1, 1).to(DEVICE)
            fake_image = gen(noise, alpha, step)
            fake_image = fake_image.cpu().numpy()
            fake_image = fake_image * 0.5 + 0.5
            fake_image = fake_image[0,0]
            fake_image = denormalize(fake_image)
            fake_image = np.around(fake_image)
            image_file = f'GAN_{hist_type}_{str(i).zfill(4)}.dcm'
            image_path = os.path.join(path, image_file)
            write_dicom(fake_image, image_path)

def dicom_to_png(dicom_file, output_dir, rescale_size=None):
    data = dcmread(dicom_file)
    image = data.pixel_array
    intercept = int(data.RescaleIntercept)
    slope = int(data.RescaleSlope)
    image = slope * image + intercept
    image = normalize(image)
    image = image * 255
    if rescale_size:
        transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((rescale_size, rescale_size)),
        transforms.ToTensor(),
        ])
        image = np.array(image).astype('float32')
        image = transform(image).numpy()[0]
    
    # Convert to uint
    image = np.uint8(image)
    shape = image.shape
    # Write the PNG file
    output_file = dicom_file.split('/')[-1].replace('dcm', 'png')
    with open(output_dir + output_file, 'wb') as png_file:
        w = png.Writer(shape[1], shape[0], greyscale=True)
        w.write(png_file, image)

0